In [1]:
import pandas as pd
import datetime
import numpy as np
from nombres import normalizacion_nombres
from nacionalidades import normalizacion_paises
from institucion import normalizacion_universidades
from funciones import limpiar_caracteres_especiales, limpiar_funciones, instrumento_financiamiento, normalizacion_ciudad, ciudad_a_region
from funciones import institucion_instrumento, descripcion_codigo_inach, region_institucion, macrozona_region, proyecto_concursable
from area_ocde import equivalencia_objetivos

### Funciones utiles

In [2]:
ruta_lectura = "procien/"
ruta_final = "procien/finales/"
lista_hojas = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

## Investigadores asociados

In [3]:
xls_base = pd.read_excel(f"{ruta_lectura}PROCIEN clean data.xlsx", sheet_name=lista_hojas)

In [ ]:
new_xls = []

for n in lista_hojas:
    tmp_xls = xls_base[n][['codigo', 'titulo', 'pi', 'ciudad', 'persona', 'inst', 'pais', 'genero']]
    
    for row in tmp_xls.to_numpy().tolist():
        codigo, titulo, pi, ciudad, persona, inst, pais, genero = row

        codigo = codigo.split( ' ' )[0].replace( '_', '-' )
        concursable =  proyecto_concursable( codigo.split('-')[0] )
        o1, o2, o3 = equivalencia_objetivos[codigo]
        proyecto, tesis, terreno = descripcion_codigo_inach( codigo )
        instrumento = instrumento_financiamiento( codigo.split('-')[0] )
        inst_instrumento = institucion_instrumento( codigo )

        titulo = limpiar_caracteres_especiales( titulo.lower() )

        inst = limpiar_caracteres_especiales( inst.lower().strip() )
        inst = normalizacion_universidades[ inst ]
        region_inst = region_institucion[inst]
        macro_inst = macrozona_region[region_inst]

        pais = normalizacion_paises[ limpiar_caracteres_especiales( pais.lower() ).strip().upper() ].lower().strip()
        persona = normalizacion_nombres[ limpiar_caracteres_especiales( persona.lower().strip() ) ]

        ciudad = normalizacion_ciudad[ limpiar_caracteres_especiales( ciudad.lower().strip() ) ]
        region_ciudad = ciudad_a_region[ciudad]
        macro_ciudad = macrozona_region[region_ciudad]

        pi = limpiar_caracteres_especiales( pi.lower() ).strip()
        if '/' in pi:
            pi = pi.split('/')[0].strip()
        elif '-' in pi:
            pi = pi.split('-')[0].strip()
        pi = normalizacion_nombres[pi]

        new_xls.append([n, codigo, proyecto, tesis, terreno, concursable, instrumento, inst_instrumento, titulo, pi, 
            inst, region_inst, macro_inst, pais, persona, ciudad, region_ciudad, macro_ciudad, o1, o2, o3]) 
        
    print(f"{n} done")

In [5]:
fecha = datetime.datetime.now()

full_dataframe = pd.DataFrame(np.array(new_xls), columns=['anio', 'codigo', 'tipo-proyecto', 'tesis', 
    'terreno', 'consursable', 'instrumento', 'inst-instrumento', 'titulo', 'pi', 'inst', 'region-inst', 'macro-inst',
    'pais', 'persona', 'ciudad-pi', 'region-pi', 'macro-pi', 'ocde', 'inach', 'anid'])
full_dataframe.to_excel(f"{ruta_final}PROCIEN todos-personas {fecha.day}.{fecha.month}.{fecha.year}.xlsx")